<a href="https://colab.research.google.com/github/MakrandDhanokar/CDAC-DAI/blob/main/CDAC_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import numpy as np
import os
import matplotlib.pyplot

from sklearn import datasets


In [8]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

N_SAMPLE = 1000
NOISE = 0.2
ALPHA = 0.1
tEST_SIZE = 0.2

inpDIr = '../../input'
outDir = '../../output'

params =

In [9]:
# sklearn make_moons
X, y = datasets.make_moons(n_samples=N_SAMPLE, noise=NOISE, shuffle= True, random_state=RANDOM_STATE)

In [10]:
X.shape, y.shape

((1000, 2), (1000,))

### EDA
1. plot the data
2. Print min, max, averages of the datasets

In [36]:
import pandas as pd
data_df = pd.DataFrame(X, columns = ['A', 'B'])
data_df['target'] = y
data_df.head()

,A,B,target
0,-0.111667,0.520224,1
1,1.142650,-0.342577,1
2,0.795558,-0.011442,1
3,0.111827,-0.551932,1
4,-0.816466,0.543996,0


In [37]:
def predict(row, wghts):
  z = wghts[0]

  for i in range(len(row)-1):
    z += wghts[i+1] * row[i]
  return 1 if z >= 0 else 0

In [38]:
weights = np.array([0.1, 2.0, 1.0])
predict(data_df.loc[0].values, weights)

1

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tEST_SIZE, stratify=y, random_state=RANDOM_STATE)

In [20]:
data_train, data_test = data_df.to_numpy()[:900], data_df.to_numpy()[900:]
display(data_train.shape, data_test.shape)
type(data_train)

(900, 3)

(100, 3)

numpy.ndarray

In [21]:
data_df

,A,B,target
0,-0.111667,0.520224,1
1,1.142650,-0.342577,1
2,0.795558,-0.011442,1
3,0.111827,-0.551932,1
4,-0.816466,0.543996,0
...,...,...,...
995,0.835830,0.435532,0
996,-0.195852,0.833794,0
997,1.727608,-0.428829,1
998,-1.014486,0.135229,0


In [27]:
# rng = np.random.default_rng(seed=RANDOM_STATE)
# weights = rng.random(data_train.shape[1])
# weights

array([0.77395605, 0.43887844, 0.85859792])

In [ ]:
rng = np.random.default_rng(seed = RANDOM_STATE)
def train_weights(train, alpha, n_epoch):
  errors = []
  weights = rng.random(train.shape[1]) # randomly generted weights
  m = train.shape[0] # number of samples

  for epoch in range(n_epoch):
    sum_error = 0.0
    for row in train:
      prediction = predict(row, weights)
      dz = prediction - row[-1]
      sum_error += (prediction - row[-1])**2
      weights[0] += alpha * dz /m
      for i in range(len(row) - 1):
        weights[i+1] += -alpha * dz * row[i] / m
    errors.append(sum_error)
  return errors, weights

## `***Deep Neural Networks***`

Assignment 2
